In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import time
import json
import pandas as pd
import sys
from tqdm import tqdm_notebook as tqdm
import copy
import os.path as osp
%run ./plot_functions.ipynb
sys.path.insert(0, '../src/')


output_path =  ../output/figures
plt = single_column_plt_params(plt)
plt = double_column_plt_params(plt)


In [2]:
from benchmark_utilities import benchmark_pnml, cat_benchmark_df
from baseline_utilities import odin_df_densnet_cifar10, odin_df_densnet_cifar100, odin_df_resnet_cifar10, odin_df_resnet_cifar100
from baseline_utilities import lvo_df_densnet_cifar10, lvo_df_densnet_cifar100, lvo_df_resnet_cifar10, lvo_df_resnet_cifar100
plt = single_column_plt_params(plt)


In [3]:
base_dir = osp.join('..', 'output', 'embedding')
ood_name_benchmark_list = ['Imagenet (crop)',
                           'Imagenet (resize)',
                           'LSUN (crop)',
                           'LSUN (resize)',
                           'iSUN', 'Uniform',
                           'Gaussian']


## Densnet CIFAR100

In [4]:
# pnml_5_aug_df_densnet_cifar100, _ = benchmark_pnml(ood_name_benchmark_list,
#                                                    'DensNet-BC CIFAR100',
#                                                    'pNML 5 Aug',
#                                                    base_dir,
#                                                    is_5_aug=True)
# pnml_5_aug_df_densnet_cifar100


In [5]:
pnml_df_densnet_cifar100, results_densnet_cifar100_dict = benchmark_pnml(
    ood_name_benchmark_list, 'DensNet-BC CIFAR100', 'pNML', base_dir)
pnml_df_densnet_cifar100


benchmark_pnml. model_name=DensNet-BC CIFAR100, is_5_aug=False, is_genie_estimate=False
Imagenet (crop)


100%|██████████| 2887/2887 [04:35<00:00,  9.03it/s]


Imagenet (resize)


100%|██████████| 2459/2459 [00:13<00:00, 176.34it/s]


LSUN (crop)


...

Uniform


100%|██████████| 2161/2161 [00:00<00:00, 2363.16it/s]


Gaussian


100%|██████████| 2158/2158 [00:00<00:00, 370530.05it/s]


,Model,OOD Datasets,Method,FPR (95% TPR) ↓,Detection Error ↓,AUROC ↑,AP-In ↑,AP-Out ↑
0,DensNet-BC CIFAR100,Imagenet (crop),pNML,63.000000,34.000000,88.404830,90.569093,84.260377
1,DensNet-BC CIFAR100,Imagenet (resize),pNML,14.990000,9.995000,97.142526,97.389129,96.824443
2,DensNet-BC CIFAR100,LSUN (crop),pNML,65.680000,35.340000,85.562111,87.279766,82.087952
3,DensNet-BC CIFAR100,LSUN (resize),pNML,10.900000,7.950000,97.792803,98.134696,97.195153
4,DensNet-BC CIFAR100,iSUN,pNML,10.633053,7.816527,97.791386,97.981297,97.482305
5,DensNet-BC CIFAR100,Uniform,pNML,0.000000,2.500000,100.000000,100.000000,100.000000
6,DensNet-BC CIFAR100,Gaussian,pNML,0.000000,2.500000,100.000000,100.000000,100.000000


In [6]:
result_densnet_cifar100 = cat_benchmark_df(odin_df_densnet_cifar100,
                                           lvo_df_densnet_cifar100,
                                           pnml_df_densnet_cifar100,
                                           pnml_df_densnet_cifar100)
                                           #pnml_5_aug_df_densnet_cifar100)
result_densnet_cifar100


,Model,OOD Datasets,Method,FPR (95% TPR) ↓,Detection Error ↓,AUROC ↑,AP-In ↑,AP-Out ↑
0,DensNet-BC CIFAR100,Imagenet (crop),ODIN,17.3,11.2,97.1,97.4,96.8
0,DensNet-BC CIFAR100,Imagenet (crop),LOO,8.3,6.3,98.4,98.6,98.3
0,DensNet-BC CIFAR100,Imagenet (crop),pNML,63.0,34.0,88.4,90.6,84.3
0,DensNet-BC CIFAR100,Imagenet (crop),pNML,63.0,34.0,88.4,90.6,84.3
1,DensNet-BC CIFAR100,Imagenet (resize),ODIN,44.3,24.6,90.7,91.4,90.1
1,DensNet-BC CIFAR100,Imagenet (resize),LOO,20.5,10.0,96.3,96.7,95.8
1,DensNet-BC CIFAR100,Imagenet (resize),pNML,15.0,10.0,97.1,97.4,96.8
1,DensNet-BC CIFAR100,Imagenet (resize),pNML,15.0,10.0,97.1,97.4,96.8
2,DensNet-BC CIFAR100,LSUN (crop),ODIN,17.6,11.3,96.8,97.1,96.5
2,DensNet-BC CIFAR100,LSUN (crop),LOO,14.7,8.5,97.4,97.6,97.2


## Densnet CIFAR10

In [7]:
# pnml_5_aug_df_densnet_cifar10, _ = benchmark_pnml(ood_name_benchmark_list,
#                                                   'DensNet-BC CIFAR10',
#                                                   'pNML 5 Aug',
#                                                   base_dir,
#                                                   is_5_aug=True)
# pnml_5_aug_df_densnet_cifar10


In [8]:
pnml_df_densnet_cifar10, results_densnet_cifar10_dict = benchmark_pnml(ood_name_benchmark_list,
                                                                       'DensNet-BC CIFAR10',
                                                                       'pNML',
                                                                       base_dir
                                                                       )
pnml_df_densnet_cifar10


benchmark_pnml. model_name=DensNet-BC CIFAR10, is_5_aug=False, is_genie_estimate=False
Imagenet (crop)


100%|██████████| 45/45 [03:50<00:00,  5.05s/it]


Imagenet (resize)


100%|██████████| 45/45 [00:00<00:00, 21418.94it/s]

LSUN (crop)


...


100%|██████████| 45/45 [00:00<00:00, 97390.96it/s]

Uniform



100%|██████████| 45/45 [00:00<00:00, 192203.34it/s]

Gaussian


,Model,OOD Datasets,Method,FPR (95% TPR) ↓,Detection Error ↓,AUROC ↑,AP-In ↑,AP-Out ↑
0,DensNet-BC CIFAR10,Imagenet (crop),pNML,0.530000,2.765000,99.524350,99.625903,99.319960
1,DensNet-BC CIFAR10,Imagenet (resize),pNML,16.280000,10.640000,97.012345,97.230502,96.855320
2,DensNet-BC CIFAR10,LSUN (crop),pNML,0.300000,2.650000,99.696182,99.753214,99.575968
3,DensNet-BC CIFAR10,LSUN (resize),pNML,10.900000,7.950000,97.863676,98.133377,97.520192
4,DensNet-BC CIFAR10,iSUN,pNML,10.711485,7.855742,97.926604,98.087668,97.732962
5,DensNet-BC CIFAR10,Uniform,pNML,0.000000,2.500000,99.996791,99.997571,99.996131
6,DensNet-BC CIFAR10,Gaussian,pNML,0.000000,2.500000,99.999993,99.999993,99.999993


In [9]:
result_densnet_cifar10 = cat_benchmark_df(odin_df_densnet_cifar10,
                                          lvo_df_densnet_cifar10,
                                          pnml_df_densnet_cifar10,pnml_df_densnet_cifar10)
#                                           pnml_5_aug_df_densnet_cifar10)
result_densnet_cifar10


,Model,OOD Datasets,Method,FPR (95% TPR) ↓,Detection Error ↓,AUROC ↑,AP-In ↑,AP-Out ↑
0,DensNet-BC CIFAR10,Imagenet (crop),ODIN,4.3,4.7,99.1,99.1,99.1
0,DensNet-BC CIFAR10,Imagenet (crop),LOO,1.2,2.6,99.6,99.7,99.6
0,DensNet-BC CIFAR10,Imagenet (crop),pNML,0.5,2.8,99.5,99.6,99.3
0,DensNet-BC CIFAR10,Imagenet (crop),pNML,0.5,2.8,99.5,99.6,99.3
1,DensNet-BC CIFAR10,Imagenet (resize),ODIN,7.5,6.3,98.5,98.6,98.5
1,DensNet-BC CIFAR10,Imagenet (resize),LOO,2.9,3.8,99.3,99.4,99.3
1,DensNet-BC CIFAR10,Imagenet (resize),pNML,16.3,10.6,97.0,97.2,96.9
1,DensNet-BC CIFAR10,Imagenet (resize),pNML,16.3,10.6,97.0,97.2,96.9
2,DensNet-BC CIFAR10,LSUN (crop),ODIN,8.7,6.9,98.2,98.5,97.8
2,DensNet-BC CIFAR10,LSUN (crop),LOO,3.4,4.1,99.2,99.3,99.2


# Resnet CIFAR 10

In [10]:
# pnml_5_aug_df_resnet_cifar10, _ = benchmark_pnml(ood_name_benchmark_list,
#                                                  'WRN-28-10 CIFAR10',
#                                                  'pNML 5 Aug',
#                                                  base_dir,
#                                                  is_5_aug=True)
# pnml_5_aug_df_resnet_cifar10


In [11]:
pnml_df_resnet_cifar10, results_resnet_cifar10_dict = benchmark_pnml(ood_name_benchmark_list,
                                                                     'WRN-28-10 CIFAR10',
                                                                     'pNML',
                                                                     base_dir)
pnml_df_resnet_cifar10


benchmark_pnml. model_name=WRN-28-10 CIFAR10, is_5_aug=False, is_genie_estimate=False
Imagenet (crop)


100%|██████████| 45/45 [07:36<00:00, 14.19s/it]


Imagenet (resize)


100%|██████████| 45/45 [00:00<00:00, 60826.19it/s]


LSUN (crop)


...

Uniform


100%|██████████| 45/45 [00:00<00:00, 64395.66it/s]


Gaussian


100%|██████████| 45/45 [00:00<00:00, 89621.88it/s]


,Model,OOD Datasets,Method,FPR (95% TPR) ↓,Detection Error ↓,AUROC ↑,AP-In ↑,AP-Out ↑
0,WRN-28-10 CIFAR10,Imagenet (crop),pNML,0.000000,2.500000,99.795775,99.847912,99.614590
1,WRN-28-10 CIFAR10,Imagenet (resize),pNML,6.770000,5.885000,98.419944,98.682880,97.922374
2,WRN-28-10 CIFAR10,LSUN (crop),pNML,0.010000,2.505000,99.843389,99.886370,99.698802
3,WRN-28-10 CIFAR10,LSUN (resize),pNML,4.790000,4.895000,98.845957,99.038382,98.485686
4,WRN-28-10 CIFAR10,iSUN,pNML,5.815126,5.407563,98.703913,98.898340,98.328800
5,WRN-28-10 CIFAR10,Uniform,pNML,0.000000,2.500000,99.508270,99.694418,98.694160
6,WRN-28-10 CIFAR10,Gaussian,pNML,0.000000,2.500000,99.944696,99.968307,99.770611


In [12]:
result_resnet_cifar10 = cat_benchmark_df(odin_df_resnet_cifar10,
                                         lvo_df_resnet_cifar10,
                                         pnml_df_resnet_cifar10,    pnml_df_resnet_cifar10)
#                                          pnml_5_aug_df_resnet_cifar10)
result_resnet_cifar10


,Model,OOD Datasets,Method,FPR (95% TPR) ↓,Detection Error ↓,AUROC ↑,AP-In ↑,AP-Out ↑
0,WRN-28-10 CIFAR10,Imagenet (crop),ODIN,23.4,14.2,94.2,92.8,94.7
0,WRN-28-10 CIFAR10,Imagenet (crop),LOO,0.8,2.2,99.8,99.8,99.8
0,WRN-28-10 CIFAR10,Imagenet (crop),pNML,0.0,2.5,99.8,99.8,99.6
0,WRN-28-10 CIFAR10,Imagenet (crop),pNML,0.0,2.5,99.8,99.8,99.6
1,WRN-28-10 CIFAR10,Imagenet (resize),ODIN,25.5,15.2,92.1,89.0,93.6
1,WRN-28-10 CIFAR10,Imagenet (resize),LOO,2.9,3.8,99.4,99.4,99.4
1,WRN-28-10 CIFAR10,Imagenet (resize),pNML,6.8,5.9,98.4,98.7,97.9
1,WRN-28-10 CIFAR10,Imagenet (resize),pNML,6.8,5.9,98.4,98.7,97.9
2,WRN-28-10 CIFAR10,LSUN (crop),ODIN,21.8,13.4,95.9,95.8,95.5
2,WRN-28-10 CIFAR10,LSUN (crop),LOO,1.9,3.2,99.6,99.6,99.6


# ResNet CIFAR100

In [13]:
# pnml_5_aug_df_resnet_cifar100, _ = benchmark_pnml(ood_name_benchmark_list,
#                                                   'WRN-28-10 CIFAR100',
#                                                   'pNML 5 Aug',
#                                                   is_5_aug=True)
# pnml_5_aug_df_resnet_cifar100


In [14]:
pnml_df_resnet_cifar100, results_resnet_cifar100_dict = benchmark_pnml(ood_name_benchmark_list,
                                                                       'WRN-28-10 CIFAR100',
                                                                       'pNML',
                                                                       base_dir)
pnml_df_resnet_cifar100


benchmark_pnml. model_name=WRN-28-10 CIFAR100, is_5_aug=False, is_genie_estimate=False
Imagenet (crop)


100%|██████████| 3913/3913 [16:21<00:00,  5.48it/s]  


Imagenet (resize)


100%|██████████| 3509/3509 [00:23<00:00, 151.72it/s]


LSUN (crop)


...

Uniform


100%|██████████| 3180/3180 [00:00<00:00, 16135.18it/s]


Gaussian


100%|██████████| 3178/3178 [00:00<00:00, 525383.24it/s]


,Model,OOD Datasets,Method,FPR (95% TPR) ↓,Detection Error ↓,AUROC ↑,AP-In ↑,AP-Out ↑
0,WRN-28-10 CIFAR100,Imagenet (crop),pNML,8.330000,6.665000,98.401321,98.624002,98.159301
1,WRN-28-10 CIFAR100,Imagenet (resize),pNML,63.600000,34.300000,87.737923,89.908400,84.654494
2,WRN-28-10 CIFAR100,LSUN (crop),pNML,17.260000,11.130000,97.096669,97.576585,96.544417
3,WRN-28-10 CIFAR100,LSUN (resize),pNML,71.900000,38.450000,86.476978,89.385137,81.909124
4,WRN-28-10 CIFAR100,iSUN,pNML,75.260504,40.130252,85.100409,88.161545,79.510567
5,WRN-28-10 CIFAR100,Uniform,pNML,99.960000,52.480000,81.318741,88.699229,66.749818
6,WRN-28-10 CIFAR100,Gaussian,pNML,96.700000,50.850000,87.441541,92.309415,74.944593


In [15]:
result_resnet_cifar100 = cat_benchmark_df(odin_df_resnet_cifar100,
                                          lvo_df_resnet_cifar100,
                                          pnml_df_resnet_cifar100,
                                          pnml_df_resnet_cifar100)
#                                           pnml_5_aug_df_resnet_cifar100)

result_resnet_cifar100


,Model,OOD Datasets,Method,FPR (95% TPR) ↓,Detection Error ↓,AUROC ↑,AP-In ↑,AP-Out ↑
0,WRN-28-10 CIFAR100,Imagenet (crop),ODIN,43.9,24.4,90.8,91.4,90.0
0,WRN-28-10 CIFAR100,Imagenet (crop),LOO,9.2,6.7,98.2,98.4,98.1
0,WRN-28-10 CIFAR100,Imagenet (crop),pNML,8.3,6.7,98.4,98.6,98.2
0,WRN-28-10 CIFAR100,Imagenet (crop),pNML,8.3,6.7,98.4,98.6,98.2
1,WRN-28-10 CIFAR100,Imagenet (resize),ODIN,55.9,30.4,84.0,82.8,84.4
1,WRN-28-10 CIFAR100,Imagenet (resize),LOO,24.5,11.6,95.2,95.5,94.8
1,WRN-28-10 CIFAR100,Imagenet (resize),pNML,63.6,34.3,87.7,89.9,84.7
1,WRN-28-10 CIFAR100,Imagenet (resize),pNML,63.6,34.3,87.7,89.9,84.7
2,WRN-28-10 CIFAR100,LSUN (crop),ODIN,39.6,22.3,92.0,92.4,91.6
2,WRN-28-10 CIFAR100,LSUN (crop),LOO,14.2,8.2,97.4,97.6,97.2


# Summary

In [16]:
summary_df = pd.concat([result_densnet_cifar10,
                        result_densnet_cifar100,
                        result_resnet_cifar10,
                        result_resnet_cifar100])  # .reset_index(drop=True)

prev_row = summary_df.iloc[-1]['OOD Datasets']
change_idx = []
len_df = len(summary_df)
for iloc_idx in range(len_df):

    rev_idx = len_df - iloc_idx - 1
    row = summary_df.iloc[rev_idx]
    if row['OOD Datasets'] != prev_row:
        empty_row = pd.Series(None, index=[rev_idx+1])
        sbummary_df = pd.concat([summary_df.iloc[:rev_idx+1], empty_row,
                                 summary_df.iloc[rev_idx+1:]]).reset_index(drop=True)
    prev_row = row['OOD Datasets']
summary_df.to_csv(osp.join('..', 'output', 'summary.csv'), index=False)


/Users/kobibi/anaconda3/envs/pt36/lib/python3.6/site-packages/pandas/core/indexes/api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


# Visualize regret

In [17]:
def sigmoid(x, derivative=False):
  return x*(1-x) if derivative else 1/(1+np.exp(-x))

In [18]:
ind_np = results_densnet_cifar100_dict['regret']['iSUN']['IND']
ood_np = results_densnet_cifar100_dict['regret']['iSUN']['OOD']
ind_inds = ind_np.argsort(axis=1)
# ind_inds
print(ind_np.shape)
print(ind_inds.shape)
m, n = ind_np.shape
out = np.take_along_axis(ind_np, ind_inds.astype(int), 1)


TypeError: 'NoneType' object is not subscriptable

In [ ]:
fig = plt.figure()
size = fig.get_size_inches()
for ood_name in results_densnet_cifar100_dict['regret'].keys():
    ind_np = results_densnet_cifar100_dict['regret'][ood_name]['IND']
    ood_np = results_densnet_cifar100_dict['regret'][ood_name]['OOD']

    ind_inds = ind_np.argsort(axis=1)
    ood_inds = ood_np.argsort(axis=1)

    ind_np = np.take_along_axis(ind_np, ind_inds.astype(int), 1)
    ood_np = np.take_along_axis(ood_np, ood_inds.astype(int), 1)

    ind_prob_np = results_densnet_cifar100_dict['prob'][ood_name]['IND']
    ood_prob_np = results_densnet_cifar100_dict['prob'][ood_name]['OOD']
    ind_prob_np = np.take_along_axis(
        sigmoid(ind_prob_np), ind_inds.astype(int), 1)
    ood_prob_np = np.take_along_axis(
        sigmoid(ood_prob_np), ood_inds.astype(int), 1)


    fig, axs = plt.subplots(1, 2, figsize=(2*size[0], size[1]))
    axs[0].bar(np.arange(0, ind_np.shape[1]), np.mean(
        ind_np, axis=0), alpha=0.8, label='In-Dist')
    axs[0].bar(np.arange(0, ood_np.shape[1]), np.mean(
        ood_np, axis=0), alpha=0.5, label='OOD')
    axs[0].legend()
    axs[0].set_title(ood_name)
    axs[0].set_xlim(0, 10)

    axs[1].bar(np.arange(0, ind_prob_np.shape[1]), np.mean(
        ind_prob_np, axis=0), alpha=0.8, label='In-Dist')
    axs[1].bar(np.arange(0, ood_prob_np.shape[1]), np.mean(
        ood_prob_np, axis=0), alpha=0.5, label='OOD')
    plt.legend()
    plt.title(ood_name)
    plt.xlim(0, 10)
    plt.show()


In [ ]:
fig = plt.figure()
size = fig.get_size_inches()
for ood_name in results_resnet_cifar100_dict['regret'].keys():
    ind_np = results_resnet_cifar100_dict['regret'][ood_name]['IND']
    ood_np = results_resnet_cifar100_dict['regret'][ood_name]['OOD']

    ind_inds = ind_np.argsort(axis=1)
    ood_inds = ood_np.argsort(axis=1)

    ind_np = np.take_along_axis(ind_np, ind_inds.astype(int), 1)
    ood_np = np.take_along_axis(ood_np, ood_inds.astype(int), 1)

    ind_prob_np = results_resnet_cifar100_dict['prob'][ood_name]['IND']
    ood_prob_np = results_resnet_cifar100_dict['prob'][ood_name]['OOD']
    ind_prob_np = np.take_along_axis(
        sigmoid(ind_prob_np), ind_inds.astype(int), 1)
    ood_prob_np = np.take_along_axis(
        sigmoid(ood_prob_np), ood_inds.astype(int), 1)

    fig, axs = plt.subplots(1, 3, figsize=(3*size[0], size[1]))
    axs[0].bar(np.arange(0, ind_np.shape[1]), np.mean(
        ind_np, axis=0), alpha=0.8, label='In-Dist')
    axs[0].bar(np.arange(0, ood_np.shape[1]), np.mean(
        ood_np, axis=0), alpha=0.5, label='OOD')
    axs[0].legend()
    axs[0].set_title(ood_name)
    axs[0].set_xlim(0, 20)

    axs[1].bar(np.arange(0, ind_prob_np.shape[1]), np.mean(
        ind_prob_np, axis=0), alpha=0.8, label='In-Dist')
    axs[1].bar(np.arange(0, ood_prob_np.shape[1]), np.mean(
        ood_prob_np, axis=0), alpha=0.5, label='OOD')

    axs[2].bar(np.arange(0, ind_prob_np.shape[1]), np.mean(
        ind_prob_np * 2 ** ind_np, axis=0), alpha=0.8, label='In-Dist')
    axs[2].bar(np.arange(0, ood_prob_np.shape[1]), np.mean(
        ood_prob_np* 2 ** ood_np, axis=0), alpha=0.5, label='OOD')
    plt.title(ood_name)
    plt.xlim(0, 20)
    plt.show()


## Deep dive pNML

In [ ]:
from distributions_metrics import calc_performance_in_out_dist


def compute_regret_deep_dive(x_m_train: np.ndarray, x_m_test: np.ndarray, lamb: float, u=None, eta=None, vh=None):
    n = x_m_train.shape[1]
    
    # SVD decomposition
    if u is None or eta is None or None or vh is None:
        u, s, vh = np.linalg.svd(x_m_train)
        eta = s ** 2

        # pad to fit u
        padded = np.zeros(u.shape[0])
        padded[:eta.shape[0]] = eta
        eta = padded

        eta = eta[:, np.newaxis]

    # Calc mean regret
    x_t_u_2 = (x_m_test.T.dot(u)) ** 2
    div = x_t_u_2 / (eta.T + lamb)
    regret_list = np.log(1 + (1 / n) * div.sum(axis=1)).tolist()
    return regret_list, u, eta, vh


In [ ]:
pnml_dataset_input = resnet_cifar10_path_dict

pnml_dataset = copy.deepcopy(pnml_dataset_input)

# Get In-Distribution datasets
trainset_path = pnml_dataset['trainset']
train_labels_path = pnml_dataset['train_labels']
testset_path = pnml_dataset['testset']
trainset_all = np.load(trainset_path).T
testset_ind = np.load(testset_path).T
train_labels = np.load(train_labels_path)

# Initialize performance dataframe output
perf_df_list = []

# Iterate over Out-Of-Distribution datasets
ood_name = 'LSUN (resize)'
npy_path = pnml_dataset[ood_name]


print('pNML for {}'.format(ood_name))
testset_ood = np.load(npy_path).T

# testset_ind -= testset_ind.mean(axis=0)
# testset_ood -= testset_ood.mean(axis=0)


lamb = 0
for lamb in [1, 1e-1, 1e-3, 1e-6, 1e-9, 1e-12, 1e-16, 1e-19]:
    regret_ind_list_all = []
    regret_ood_list_all = []
    for class_num in tqdm(np.unique(train_labels)):
        trainset = trainset_all[:, train_labels == class_num]
    #     trainset -= trainset.mean(axis=0)

        regret_ind_list, u, eta, vh = compute_regret_deep_dive(
            trainset, testset_ind, lamb=lamb, u=None, eta=None, vh=None)
        regret_ood_list, _, _, _ = compute_regret_deep_dive(
            trainset, testset_ood, lamb=lamb, u=u, eta=eta, vh=vh)

        regret_ind_list_all.append(regret_ind_list)
        regret_ood_list_all.append(regret_ood_list)

    regret_ind_np = np.asarray(regret_ind_list_all)
    regret_ood_np = np.asarray(regret_ood_list_all)

    regret_ind_min_list = np.min(regret_ind_np, axis=0).tolist()
    regret_ood_min_list = np.min(regret_ood_np, axis=0).tolist()

    assert isinstance(regret_ind_min_list, list)
    assert isinstance(regret_ood_min_list, list)

    y_score_ind = regret_ind_min_list + regret_ood_min_list
    y_score_ind = 1 - np.array(y_score_ind)
    y_true_ind = [True] * len(regret_ind_min_list) + \
        [False] * len(regret_ood_min_list)
    performance = calc_performance_in_out_dist(y_true_ind, y_score_ind)
    print('lamb: {} {}'.format(lamb, float(performance['AUROC ↑'])))


In [ ]:
print('Vanilla')
performance


In [ ]:
x_m_train = trainset
x_m_test = testset_ood[:, 0]
u = u
eta = eta
vh = vh
dim = dim
lamb = lamb
n = x_m_train.shape[1]

# SVD decomposition
if u is None or eta is None or None or vh is None:
    u, s, vh = np.linalg.svd(x_m_train)
    eta = s ** 2

    # pad to fit u
    padded = np.zeros(u.shape[0])
    padded[:eta.shape[0]] = eta
    eta = padded

    eta = eta[:, np.newaxis]

# Calc mean regret
x_t_u_2_ood = (x_m_test.T.dot(u)) ** 2
div_ood = x_t_u_2_ood / (eta.T + lamb)

regret_ood_list = np.log(1 + (1 / n) * div_ood.sum(axis=1)
                         ).tolist()  # div.sum(axis=1).tolist()


In [ ]:
x_m_train = trainset
x_m_test = testset_ind[:, 0]
u = u
eta = eta
vh = vh
dim = dim
lamb = 0#lamb
n = x_m_train.shape[1]

# SVD decomposition
if u is None or eta is None or None or vh is None:
    u, s, vh = np.linalg.svd(x_m_train)
    eta = s ** 2

    # pad to fit u
    padded = np.zeros(u.shape[0])
    padded[:eta.shape[0]] = eta
    eta = padded

    eta = eta[:, np.newaxis]

# Calc mean regret
x_t_u_2_ind = (x_m_test.T.dot(u)) ** 2
div_ind = x_t_u_2_ind / (eta.T + lamb)

# div_ind[div_ind == np.inf] = 10e12
# div_ind[div_ind==np.nan] = 10e12

# np.log(1 + (1 / n) * div.sum(axis=1)).tolist()
regret_ind_list = np.log(1 + (1 / n) * div_ind.sum(axis=1)).tolist() #div.sum(axis=1).tolist()


In [ ]:
plt.plot(x_t_u_2_ind)
plt.plot(x_t_u_2_ood)
plt.yscale('symlog')
plt.show()

In [ ]:
_,bins,_ = plt.hist(regret_ind_min_list,label='In')#, range=(0,1))
plt.hist(regret_ood_min_list,bins=bins,alpha=0.5,label='Out')
plt.legend()
plt.show()
